In [8]:
import time
import datetime
import requests
import pandas as pd
import numpy as np

In [9]:
# Get all available exchanges from cryptocompare.com as a json format
def get_all_exchanges(apikey):
    # Config data volumn parm
    payload_volumn = {
            "api_key": apikey,
            }

    # Get data total volumn
    # Use ''/exchanges' to get exchanges list dataset
    url_vol = "https://min-api.cryptocompare.com/data/v4/all/exchanges"

    # Request data
    result_volumn = requests.get(url_vol, params=payload_volumn).json()
    return result_volumn['Data']['exchanges']

In [15]:
# Convert json data into pandas dataframe
def convert_json_to_dataframe(all_exchanges):
    ex_data = pd.DataFrame(columns=('exchange','fsym','tsym','start','end'))
    i = 0
    # The following loops are used to parse and filter the data from json
    for key, value in all_exchanges.items():
        if value['isActive'] == True:
            ex_name = key
            for key, value in value['pairs'].items():
                fsym = key
                for key, vlaue in value['tsyms'].items():
                    tsym = key
                    try: 
                        start = value['tsyms'][key]['histo_minute_start_ts']
                        end = value['tsyms'][key]['histo_minute_end_ts']
                    except BaseException as e:
                        start = ''
                        end = ''
                    # Get one exchange info
                    ex_data =  ex_data.append([{'exchange':ex_name, 'fsym': fsym,'tsym':tsym, 'start':start, 'end':end}], ignore_index=True)
    return ex_data

In [11]:
# Drop some of exchanges which has no data recorded
def drop_empty_values(data):
    # Inorder to drop null row, replace empty value as nan,
    # otherwise pandas do not recognise "" (a empty string which occur in dataset) as nan
    data.replace(to_replace=r'^\s*$',value=np.nan,regex=True,inplace=True)
    data.dropna(axis=0, how='any', inplace=True)
    return data

In [18]:
# Main function
# Get all available exchanges data, also reformat and clean data

# Use your ApiKey
apikey = '1bd6b034f5c37235fe86bd57125805ae1164b1f50aea15648073d27a3627b00e'

# Get json format
ex_json = get_all_exchanges(apikey)

# Get dataframe
ex_data = convert_json_to_dataframe(ex_json)

# Clean data
ex_data = drop_empty_values(ex_data)

# Output .csv file
ex_data.to_csv('all_ex2.csv')